# Démonstration Robo-Advisor Goal-Based

Ce notebook présente l'optimisation de portefeuille goal-based pour un robo-advisor.

In [ ]:
import sys
import os
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from data_fetcher import get_historical_data, calculate_returns, calculate_stats
from portfolio_optimizer import optimize_portfolio
from monte_carlo import monte_carlo_simulation, calculate_success_probability

## 1. Récupération des données

In [ ]:
# Définition des actifs
tickers = ['SPY', 'BND', 'GLD', 'QQQ']

# Récupération des données
data = get_historical_data(tickers)
returns = calculate_returns(data)
stats = calculate_stats(returns)

print("Statistiques des actifs:")
for ticker in tickers:
    print(f"{ticker}: Rendement annuel = {stats['expected_return'][ticker]:.2%}, Volatilité = {stats['volatility'][ticker]:.2%}")

## 2. Optimisation de portefeuille

In [ ]:
# Préparation des données pour l'optimisation
expected_returns = np.array([stats['expected_return'][t] for t in tickers])
cov_matrix = returns.cov().values * 252

# Optimisation
result = optimize_portfolio(expected_returns, cov_matrix, risk_tolerance=0.1)

print("Allocation optimale:")
for i, ticker in enumerate(tickers):
    print(f"{ticker}: {result['weights'][i]:.2%}")
print(f"\nRendement attendu: {result['expected_return']:.2%}")
print(f"Volatilité: {result['volatility']:.2%}")
print(f"Ratio de Sharpe: {result['sharpe_ratio']:.2f}")

## 3. Simulations Monte Carlo

In [ ]:
# Simulation pour un objectif
initial_investment = 50000
horizon_years = 10
target_amount = 100000

final_values = monte_carlo_simulation(expected_returns, cov_matrix, initial_investment, horizon_years)
success_prob = calculate_success_probability(final_values, target_amount)

print(f"Probabilité de succès: {success_prob:.2%}")
print(f"Valeur médiane finale: {np.median(final_values):.2f}€")
print(f"Valeur moyenne finale: {np.mean(final_values):.2f}€")

# Visualisation
plt.figure(figsize=(10, 6))
plt.hist(final_values, bins=50, alpha=0.7, edgecolor='black')
plt.axvline(target_amount, color='red', linestyle='--', label=f'Objectif: {target_amount}€')
plt.axvline(np.median(final_values), color='green', linestyle='--', label=f'Médiane: {np.median(final_values):.0f}€')
plt.xlabel('Valeur finale (€)')
plt.ylabel('Fréquence')
plt.title('Distribution des valeurs finales (Monte Carlo)')
plt.legend()
plt.show()